# Amazon Bedrock AgentCore Runtime で OpenAI モデルを使用した Strands エージェントをホスティング

## 概要

このチュートリアルでは、Amazon Bedrock AgentCore Runtime を使用して既存のエージェントをホスティングする方法を学びます。

OpenAI モデルを使用した Strands Agents の例に焦点を当てます。Amazon Bedrock モデルを使用した Strands Agents については[こちら](../01-strands-with-bedrock-model)、Amazon Bedrock モデルを使用した LangGraph については[こちら](../02-langgraph-with-bedrock-model)をご確認ください。


### チュートリアル詳細

| 情報             | 詳細                                                                      |
|:-----------------|:-------------------------------------------------------------------------|
| チュートリアルタイプ | 会話形式                                                                 |
| エージェントタイプ  | シングル                                                                  |
| エージェントフレームワーク | Strands Agents                                                     |
| LLM モデル        | GPT 4.1 mini                                                              |
| チュートリアル構成  | AgentCore Runtime でのエージェントホスティング。Strands Agent と OpenAI モデルの使用 |
| チュートリアル分野  | クロスバーティカル                                                         |
| 例の複雑さ        | 簡単                                                                      |
| 使用 SDK         | Amazon BedrockAgentCore Python SDK と boto3                               |

### チュートリアルアーキテクチャ

このチュートリアルでは、既存のエージェントを AgentCore Runtime にデプロイする方法を説明します。

デモンストレーション目的で、Amazon Bedrock モデルを使用した Strands Agent を使用します。

この例では、`get_weather` と `get_time` の2つのツールを持つ非常にシンプルなエージェントを使用します。

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="60%"/>
</div>

### チュートリアルの主な機能

* Amazon Bedrock AgentCore Runtime でのエージェントホスティング
* OpenAI モデルの使用
* Strands Agents の使用

## 前提条件

このチュートリアルを実行するには、以下が必要です：
* Python 3.10+
* AWS 認証情報
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker が実行中

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## エージェントの作成とローカルでの実験

AgentCore Runtime にエージェントをデプロイする前に、実験目的でローカルで開発・実行してみましょう。

本番用エージェントアプリケーションでは、エージェントの作成プロセスと呼び出しプロセスを分離する必要があります。AgentCore Runtime では、エージェントの呼び出し部分を `@app.entrypoint` デコレーターで装飾し、ランタイムのエントリーポイントとして使用します。まず、実験フェーズ中に各エージェントがどのように開発されるかを見てみましょう。

ここでのアーキテクチャは以下のようになります：

<div style="text-align:left">
    <img src="images/architecture_local.png" width="60%"/>
</div>

In [ ]:
%%writefile strands_agents_openai.py
from strands import Agent, tool
from strands_tools import calculator # calculator ツールをインポート
import argparse
import json
from strands.models.litellm import LiteLLMModel
import os

os.environ["AZURE_API_KEY"] = "<YOUR_API_KEY>"
os.environ["AZURE_API_BASE"] = "<YOUR_API_BASE>"
os.environ["AZURE_API_VERSION"] = "<YOUR_API_VERSION>"

# カスタムツールを作成 
@tool
def weather():
    """ 天気を取得 """ # ダミー実装
    return "sunny"

model = "azure/gpt-4.1-mini"
litellm_model = LiteLLMModel(
    model_id=model, params={"max_tokens": 32000, "temperature": 0.7}
)


agent = Agent(
    model=litellm_model,
    tools=[calculator, weather],
    system_prompt="あなたは親切なアシスタントです。簡単な数学計算ができ、天気を教えることができます。"
)

def strands_agent_open_ai(payload):
    """
    ペイロードでエージェントを呼び出す
    """
    user_input = payload.get("prompt")
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_agent_open_ai(json.loads(args.payload))
    print(response)

#### ローカルエージェントの呼び出し

In [ ]:
!python strands_agents_openai.py '{"prompt": "今の天気はどうですか？"}'

## AgentCore Runtime へのデプロイ用にエージェントを準備

それでは、エージェントを AgentCore Runtime にデプロイしましょう。そのためには以下が必要です：
* `from bedrock_agentcore.runtime import BedrockAgentCoreApp` で Runtime App をインポート
* コード内で `app = BedrockAgentCoreApp()` でアプリを初期化
* 呼び出し関数を `@app.entrypoint` デコレーターで装飾
* `app.run()` で AgentCoreRuntime にエージェントの実行を制御させる

### OpenAI モデルを使用した Strands Agents
GPT 4.1 mini モデルを使用する Strands Agent から始めましょう。他のエージェントもまったく同じように動作します。

In [ ]:
%%writefile strands_agents_openai.py
from strands import Agent, tool
from strands_tools import calculator # calculator ツールをインポート
import argparse
import json
from strands.models.litellm import LiteLLMModel
import os
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

os.environ["AZURE_API_KEY"] = "<YOUR_API_KEY>"
os.environ["AZURE_API_BASE"] = "<YOUR_API_BASE>"
os.environ["AZURE_API_VERSION"] = "<YOUR_API_VERSION>"

# カスタムツールを作成 
@tool
def weather():
    """ 天気を取得 """ # ダミー実装
    return "sunny"

model = "azure/gpt-4.1-mini"
litellm_model = LiteLLMModel(
    model_id=model, params={"max_tokens": 32000, "temperature": 0.7}
)


agent = Agent(
    model=litellm_model,
    tools=[calculator, weather],
    system_prompt="あなたは親切なアシスタントです。簡単な数学計算ができ、天気を教えることができます。"
)


@app.entrypoint
def strands_agent_open_ai(payload):
    """
    ペイロードでエージェントを呼び出す
    """
    user_input = payload.get("prompt")
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

## 裏で何が起きているのか？

`BedrockAgentCoreApp` を使用すると、以下が自動的に行われます：

* ポート 8080 でリッスンする HTTP サーバーを作成
* エージェントの要件を処理するために必要な `/invocations` エンドポイントを実装
* ヘルスチェック用の `/ping` エンドポイントを実装（非同期エージェントにとって非常に重要）
* 適切なコンテンツタイプとレスポンス形式を処理
* AWS 標準に従ったエラーハンドリングを管理

## エージェントを AgentCore Runtime にデプロイ

`CreateAgentRuntime` 操作は包括的な設定オプションをサポートしており、コンテナイメージ、環境変数、暗号化設定を指定できます。また、クライアントがエージェントとどのように通信するかを制御するために、プロトコル設定（HTTP、MCP）と認可メカニズムを設定することもできます。

**注意：** 運用のベストプラクティスは、CI/CD パイプラインと IaC を使用してコードをコンテナとしてパッケージ化し、ECR にプッシュすることです

このチュートリアルでは、Amazon Bedrock AgentCode Python SDK を使用して、アーティファクトを簡単にパッケージ化し、AgentCore Runtime にデプロイできます。

### AgentCore Runtime デプロイメントの設定

次に、スターターツールキットを使用して、エントリーポイント、先ほど作成した実行ロール、requirements ファイルで AgentCore Runtime デプロイメントを設定します。また、起動時に Amazon ECR リポジトリを自動作成するようにスターターキットを設定します。

設定ステップ中、アプリケーションコードに基づいて Docker ファイルが生成されます

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import boto3
import json
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()

agent_name = "strands_openai_getting_started"
response = agentcore_runtime.configure(
    entrypoint="strands_agents_openai.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

### エージェントを AgentCore Runtime に起動

Docker ファイルができたので、エージェントを AgentCore Runtime に起動しましょう。これにより、Amazon ECR リポジトリと AgentCore Runtime が作成されます

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()

### AgentCore Runtime のステータス確認
AgentCore Runtime をデプロイしたので、デプロイメントステータスを確認しましょう

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### AgentCore Runtime の呼び出し

最後に、ペイロードを使用して AgentCore Runtime を呼び出すことができます

<div style="text-align:left">
    <img src="images/invoke.png" width=85%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "こんにちは、何ができますか？"})
invoke_response

### 呼び出し結果の処理

呼び出し結果を処理して、アプリケーションに含めることができます

In [ ]:
from IPython.display import Markdown, display
import json
response_text = invoke_response['response'][0]
display(Markdown(response_text))

### boto3 を使用した AgentCore Runtime の呼び出し

AgentCore Runtime が作成されたので、任意の AWS SDK を使用して呼び出すことができます。たとえば、boto3 の `invoke_agent_runtime` メソッドを使用できます。

In [ ]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "2x2 はいくらですか？"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

## クリーンアップ（オプション）

作成した AgentCore Runtime をクリーンアップしましょう

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# おめでとうございます！